# 6d reconstruction scan

## define the diagnostic

In [ ]:
import pandas as pd

from plugins.interfaces.diagnostics import EPICSImageDiagnostic, ROI

screen_name = "13ARV1"
save_image_location = ""
dyg14_roi = ROI(
    xmin=0, xmax=100, ymin=0, ymax=100
)
dyg15_roi = ROI(
    xmin=0, xmax=100, ymin=0, ymax=100
)

DYG14_IP = 1
DYG15_IP = 2
charge_pvs = []

DYG14 = EPICSImageDiagnostic(
    screen_name=screen_name,
    resolution_suffix=None,
    roi=dyg14_roi,
    extra_pvs=charge_pvs,
    save_image_location=save_image_location
)

DYG15 = EPICSImageDiagnostic(
    screen_name=screen_name,
    resolution_suffix=None,
    roi=dyg15_roi,
    extra_pvs=charge_pvs,
    save_image_location=save_image_location
)

def set_camera(ip_address):
    # stop the current camera
    caput("13ARV1:cam1:Acquire", 0)
    # set the new camera IP address
    caput("13ARV1:cam1:SetCameraIP", ip_address)
    # start the new camera
    caput("13ARV1:cam1:Acquire", 1)



In [ ]:
set_camera(DYG14_IP)
DYG14.test_measurement()

In [ ]:
set_camera(DYG15_IP)
DYG15.test_measurement()

### Define the evaluator

In [1]:
from epics import caput, caget
from xopt import Evaluator
import time
import numpy as np

def evaluate_function(inputs: dict, diagnostic) -> dict:
    # caput values
    for name, val in inputs:
        caput(name, val)

    # wait for changes to occur
    time.sleep(2)

    results = diagnostic.measure_beamsize(5, fit_image=False, **inputs)

    return results

evaluator = Evaluator(function=evaluate_function)

### Set up scan

In [6]:
# quad strength pv values
x = np.linspace(0,10,5)
quad_pv_name = "name"
tdc_pv_name = "tdc_name"
dipole_pv_name = "dipole_name"
tdc_vals = {"OFF":0, "ON":1} # off/on values
dipole_vals = {"OFF":0, "ON":1} # off/on vals

def do_quad_scan(k, pv_dict, diagnostic):
    q_scan_data = []
    for ele in k:
        # add tcav/dipole settings to quad settings
        total_pvs = pv_dict | {quad_pv_name:ele}

        # do measurement
        result = evaluate_function(total_pvs, diagnostic)
        q_scan_data += [result]

    return q_scan_data

data = []

# do quad scan with dipole off
set_camera(DYG14_IP)
for tdc_state in ["OFF","ON"]:
    q_scan_results = do_quad_scan(
        x,
        {tdc_pv_name: tdc_state, dipole_pv_name: dipole_vals["OFF"]},
        DYG14
    )
    data = data + q_scan_results

# do quad scan with dipole on
set_camera(DYG15_IP)
for tdc_state in ["OFF","ON"]:
    q_scan_results = do_quad_scan(
        x,
        {tdc_pv_name: tdc_state, dipole_pv_name: dipole_vals["ON"]},
        DYG14
    )
    data = data + q_scan_results

data = pd.DataFrame(data)



,x1,x2,x3
0,0.0,0.0,0.000000
1,0.0,0.0,0.111111
2,0.0,0.0,0.222222
3,0.0,0.0,0.333333
4,0.0,0.0,0.444444
...,...,...,...
995,1.0,1.0,0.555556
996,1.0,1.0,0.666667
997,1.0,1.0,0.777778
998,1.0,1.0,0.888889


In [ ]:
import yaml

# save dataframe to file
with open('data.yml', 'w') as outfile:
    yaml.dump(data.to_dict(orient="records"), outfile, default_flow_style=False)